<a href="https://colab.research.google.com/github/stesalps/pmcp/blob/main/Google_Manus_System_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Google Manus System (Complete Version)

A powerful AI assistant system that leverages Google's local models in Colab for code generation, execution, research, and web deployment, with human-in-the-loop capabilities.

## Features

- 🧠 Uses Google's local models (Gemini, Gemma) with fallback to Ollama
- 💻 Code generation, execution, and review
- 🔍 Research and information retrieval
- 🌐 Web deployment with FRP tunneling
- 👤 Human-in-the-loop review system
- 🛠️ Extensible tool system

## How to Use

1. Run Box 1 to set up the environment
2. Run Box 2 to initialize the model system
3. Run Box 3 to set up the tool registry
4. Run Box 4 to launch the user interface
5. Run Box 5 to set up web deployment
6. Run Box 6 to enable human-in-the-loop capabilities

Let's get started!

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🔧 BOX 1: Environment Setup and Configuration - v1.0                                                     ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - System initialization and dependency installation                                                      ║
# ║ - Configuration management and directory structure setup                                                 ║
# ║ - Environment detection (Colab, local)                                                                   ║
# ║ - Logging and error handling                                                                             ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🔧 BOX 1: Initializing Environment Setup and Configuration...")

import os
import sys
import json
import time
import logging
import subprocess
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Union, Callable

# Check if running in Colab
try:
    import google.colab
    IS_COLAB = True
    print("✅ Running in Google Colab environment")
except ImportError:
    IS_COLAB = False
    print("ℹ️ Running in local environment")

# Define base directories
if IS_COLAB:
    # Check if Google Drive is mounted
    if not os.path.exists("/content/drive"):
        from google.colab import drive
        drive.mount('/content/drive')
        print("✅ Google Drive mounted")
    
    # Use Google Drive for persistence if available
    BASE_DIR = Path("/content/drive/MyDrive/GoogleManusSystem")
    if not BASE_DIR.exists():
        BASE_DIR.mkdir(parents=True, exist_ok=True)
        print(f"✅ Created base directory at {BASE_DIR}")
    
    # Local working directory for temporary files
    WORKING_DIR = Path("/content/GoogleManusSystem")
else:
    # Local environment setup
    BASE_DIR = Path(os.getcwd()) / "GoogleManusSystem"
    WORKING_DIR = BASE_DIR

# Create necessary subdirectories
WORKSPACE_DIR = BASE_DIR / "workspace"
CONFIG_DIR = BASE_DIR / "config"
LOGS_DIR = BASE_DIR / "logs"
TOOLS_DIR = BASE_DIR / "tools"
SITES_DIR = BASE_DIR / "sites"
HUMAN_REVIEW_DIR = BASE_DIR / "human_review"

for directory in [WORKSPACE_DIR, CONFIG_DIR, LOGS_DIR, TOOLS_DIR, SITES_DIR, HUMAN_REVIEW_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"✅ Ensured directory exists: {directory}")

# Setup logging
LOG_FILE = LOGS_DIR / "manus_log.json"
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("GoogleManus")

# Function to log events in JSON format
def log_event(event_type: str, details: Dict[str, Any]) -> None:
    """Log an event to the JSON log file."""
    event = {
        "timestamp": datetime.now().isoformat(),
        "type": event_type,
        **details
    }
    
    # Append to log file
    with open(LOG_FILE, "a") as f:
        f.write(json.dumps(event) + "\n")
    
    # Also log to console
    logger.info(f"{event_type}: {details}")

# Install required dependencies
required_packages = [
    "fastapi",
    "uvicorn",
    "pydantic",
    "requests",
    "nest_asyncio",
    "ipywidgets",
    "beautifulsoup4",
    "markdown",
    "python-multipart",
    "aiohttp",
    "websockets"
]

print("📦 Installing required packages...")
for package in required_packages:
    try:
        __import__(package.split("==")[0].strip())
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"⏳ Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ Installed {package}")

# Apply nest_asyncio for Jupyter compatibility
import nest_asyncio
nest_asyncio.apply()
print("✅ Applied nest_asyncio for Jupyter compatibility")

# Create or load configuration
config_file = CONFIG_DIR / "system_config.json"
if config_file.exists():
    with open(config_file, "r") as f:
        config = json.load(f)
    print("✅ Loaded existing configuration")
else:
    # Default configuration
    config = {
        "version": "1.0.0",
        "created_at": datetime.now().isoformat(),
        "updated_at": datetime.now().isoformat(),
        "default_model": "google/gemini-2.5-pro",
        "default_model_type": "google",
        "workspace_path": str(WORKSPACE_DIR),
        "log_path": str(LOG_FILE),
        "sites_path": str(SITES_DIR),
        "human_review_path": str(HUMAN_REVIEW_DIR),
        "is_colab": IS_COLAB,
        "human_review_enabled": True,
        "auto_approve_threshold": 0.8,
        "public_url": None,
        "dashboard_url": None
    }
    
    with open(config_file, "w") as f:
        json.dump(config, f, indent=2)
    print("✅ Created new configuration")

# Export configuration for other boxes
box1_exports = {
    "BASE_DIR": str(BASE_DIR),
    "WORKSPACE_DIR": str(WORKSPACE_DIR),
    "CONFIG_DIR": str(CONFIG_DIR),
    "LOGS_DIR": str(LOGS_DIR),
    "TOOLS_DIR": str(TOOLS_DIR),
    "SITES_DIR": str(SITES_DIR),
    "HUMAN_REVIEW_DIR": str(HUMAN_REVIEW_DIR),
    "LOG_FILE": str(LOG_FILE),
    "IS_COLAB": IS_COLAB,
    "config": config
}

box1_exports_file = CONFIG_DIR / "box1_exports.json"
with open(box1_exports_file, "w") as f:
    json.dump(box1_exports, f, indent=2)

print("\n🎉 BOX 1 SETUP COMPLETED SUCCESSFULLY!")
print(f"📁 Base Directory: {BASE_DIR}")
print(f"💾 Configuration: {config_file}")
print(f"📝 Log File: {LOG_FILE}")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🧠 BOX 2: Model Integration and Core Services - v1.0                                                     ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - Google model integration (Gemini, Gemma)                                                               ║
# ║ - Ollama model integration (fallback)                                                                    ║
# ║ - Model selection and configuration                                                                      ║
# ║ - Streaming response handling                                                                            ║
# ║ - Context and memory management                                                                          ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🧠 BOX 2: Initializing Model Integration and Core Services...")

import os
import sys
import json
import time
import asyncio
import requests
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable, Generator, Tuple

# Load configuration from Box 1
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    # Extract paths and configuration
    BASE_DIR = Path(box1_exports["BASE_DIR"])
    CONFIG_DIR = Path(box1_exports["CONFIG_DIR"])
    IS_COLAB = box1_exports["IS_COLAB"]
    config = box1_exports["config"]
    
    print("✅ Loaded configuration from Box 1")
except Exception as e:
    print(f"❌ Error loading Box 1 configuration: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if Box 1 hasn't been run
    BASE_DIR = Path("./GoogleManusSystem")
    CONFIG_DIR = BASE_DIR / "config"
    IS_COLAB = True
    config = {
        "default_model": "google/gemini-2.5-pro",
        "default_model_type": "google",
        "human_review_enabled": True,
        "auto_approve_threshold": 0.8
    }

# Import Google Colab AI module
try:
    from google.colab import ai
    GOOGLE_AI_AVAILABLE = True
    print("✅ Google Colab AI module imported successfully")
except ImportError:
    GOOGLE_AI_AVAILABLE = False
    print("⚠️ Google Colab AI module not available. Will use fallback mode.")

# Check for Ollama availability
OLLAMA_AVAILABLE = False
OLLAMA_URL = "http://localhost:11434/api/generate"
try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        OLLAMA_AVAILABLE = True
        print("✅ Ollama is available")
except:
    print("⚠️ Ollama is not available. Will use Google models only.")

# Get available Google models
if GOOGLE_AI_AVAILABLE:
    try:
        google_models = ai.list_models()
        print(f"✅ Available Google models: {google_models}")
    except Exception as e:
        print(f"❌ Error listing Google models: {e}")
        google_models = [
            "google/gemini-2.0-flash",
            "google/gemini-2.0-flash-lite",
            "google/gemini-2.5-flash",
            "google/gemini-2.5-flash-lite",
            "google/gemini-2.5-pro",
            "google/gemma-3-12b",
            "google/gemma-3-1b",
            "google/gemma-3-27b",
            "google/gemma-3-4b"
        ]
else:
    # Fallback model list
    google_models = [
        "google/gemini-2.0-flash",
        "google/gemini-2.0-flash-lite",
        "google/gemini-2.5-flash",
        "google/gemini-2.5-flash-lite",
        "google/gemini-2.5-pro",
        "google/gemma-3-12b",
        "google/gemma-3-1b",
        "google/gemma-3-27b",
        "google/gemma-3-4b"
    ]

# Get available Ollama models
ollama_models = []
if OLLAMA_AVAILABLE:
    try:
        response = requests.get("http://localhost:11434/api/tags")
        if response.status_code == 200:
            ollama_models = [model["name"] for model in response.json().get("models", [])]
            print(f"✅ Available Ollama models: {ollama_models}")
    except Exception as e:
        print(f"❌ Error listing Ollama models: {e}")
        ollama_models = ["llama3", "mistral", "gemma"]

# Set default model
default_model = config.get("default_model", "google/gemini-2.5-pro")
default_model_type = config.get("default_model_type", "google")

if default_model_type == "google" and default_model not in google_models:
    default_model = google_models[0] if google_models else None
    print(f"⚠️ Default Google model not available. Using {default_model} instead.")
elif default_model_type == "ollama" and default_model not in ollama_models:
    default_model = ollama_models[0] if ollama_models else None
    print(f"⚠️ Default Ollama model not available. Using {default_model} instead.")

# If no models are available, set fallback mode
if not GOOGLE_AI_AVAILABLE and not OLLAMA_AVAILABLE:
    print("⚠️ No AI models available. Using fallback mode.")
    default_model_type = "fallback"
    default_model = "fallback"
elif not default_model:
    if GOOGLE_AI_AVAILABLE:
        default_model_type = "google"
        default_model = google_models[0]
    elif OLLAMA_AVAILABLE:
        default_model_type = "ollama"
        default_model = ollama_models[0]

print(f"✅ Using default model type: {default_model_type}")
print(f"✅ Using default model: {default_model}")

# Model adapter for unified access to Google and Ollama models
class ModelAdapter:
    def __init__(self, default_model_type="google", default_google_model=None, default_ollama_model=None):
        self.default_model_type = default_model_type
        self.default_google_model = default_google_model or (google_models[0] if google_models else None)
        self.default_ollama_model = default_ollama_model or (ollama_models[0] if ollama_models else None)
        self.ollama_url = "http://localhost:11434/api/generate"
    
    async def generate_response(self, prompt: str, model_type: str = None, model_name: str = None, 
                               stream: bool = False) -> Union[str, Generator, Tuple[str, float]]:
        """Generate a response using the specified model.
        
        Args:
            prompt: The input prompt
            model_type: The type of model to use (google, ollama, or fallback)
            model_name: The specific model name
            stream: Whether to stream the response
            
        Returns:
            Generated text or a generator for streaming, with confidence score
        """
        model_type = model_type or self.default_model_type
        
        if model_type == "google" and GOOGLE_AI_AVAILABLE:
            model_name = model_name or self.default_google_model
            return await self._generate_google_response(prompt, model_name, stream)
        elif model_type == "ollama" and OLLAMA_AVAILABLE:
            model_name = model_name or self.default_ollama_model
            return await self._generate_ollama_response(prompt, model_name, stream)
        else:
            # Fallback to available model
            if GOOGLE_AI_AVAILABLE:
                return await self._generate_google_response(prompt, self.default_google_model, stream)
            elif OLLAMA_AVAILABLE:
                return await self._generate_ollama_response(prompt, self.default_ollama_model, stream)
            else:
                return await self._generate_fallback_response(prompt, stream)
    
    async def _generate_google_response(self, prompt: str, model_name: str, stream: bool = False) -> Tuple[Union[str, Generator], float]:
        """Generate a response using Google's models."""
        try:
            # Log the request
            request_id = int(time.time() * 1000)
            log_data = {
                "request_id": request_id,
                "model_type": "google",
                "model": model_name,
                "prompt_length": len(prompt),
                "streaming": stream
            }
            
            # Write to log file
            log_file = Path(box1_exports["LOGS_DIR"]) / "model_requests.jsonl"
            with open(log_file, "a") as f:
                f.write(json.dumps(log_data) + "\n")
            
            # Generate text
            if stream:
                response_stream = ai.generate_text(prompt, model_name=model_name, stream=True)
                # For streaming, we return a fixed confidence
                return response_stream, 0.9
            else:
                response = ai.generate_text(prompt, model_name=model_name)
                # For non-streaming, we return a fixed confidence
                return response, 0.9
        except Exception as e:
            error_msg = f"Error generating text with Google model: {str(e)}"
            print(f"❌ {error_msg}")
            return error_msg, 0.0
    
    async def _generate_ollama_response(self, prompt: str, model_name: str, stream: bool = False) -> Tuple[Union[str, Generator], float]:
        """Generate a response using Ollama."""
        try:
            # Log the request
            request_id = int(time.time() * 1000)
            log_data = {
                "request_id": request_id,
                "model_type": "ollama",
                "model": model_name,
                "prompt_length": len(prompt),
                "streaming": stream
            }
            
            # Write to log file
            log_file = Path(box1_exports["LOGS_DIR"]) / "model_requests.jsonl"
            with open(log_file, "a") as f:
                f.write(json.dumps(log_data) + "\n")
            
            # Prepare the request
            payload = {
                "model": model_name,
                "prompt": prompt,
                "stream": stream
            }
            
            if stream:
                # Streaming implementation
                async def stream_response():
                    response = requests.post(self.ollama_url, json=payload, stream=True)
                    for line in response.iter_lines():
                        if line:
                            data = json.loads(line)
                            yield data.get("response", "")
                
                return stream_response(), 0.8
            else:
                # Non-streaming implementation
                response = requests.post(self.ollama_url, json=payload)
                response_json = response.json()
                return response_json.get("response", ""), 0.8
        except Exception as e:
            error_msg = f"Error generating text with Ollama model: {str(e)}"
            print(f"❌ {error_msg}")
            return error_msg, 0.0
    
    async def _generate_fallback_response(self, prompt: str, stream: bool = False) -> Tuple[Union[str, Generator], float]:
        """Generate a fallback response when no models are available."""
        if stream:
            async def fallback_generator():
                message = f"[FALLBACK MODE] No AI models available. Would respond to: {prompt[:100]}..."
                for char in message:
                    yield char
                    await asyncio.sleep(0.01)
            
            return fallback_generator(), 0.0
        else:
            return f"[FALLBACK MODE] No AI models available. Would respond to: {prompt[:100]}...", 0.0
    
    def list_available_models(self):
        """List all available models."""
        models = []
        
        if GOOGLE_AI_AVAILABLE:
            models.extend([{"name": model, "type": "google"} for model in google_models])
        
        if OLLAMA_AVAILABLE:
            models.extend([{"name": model, "type": "ollama"} for model in ollama_models])
        
        return models

# Initialize the model adapter
model_adapter = ModelAdapter(
    default_model_type=default_model_type,
    default_google_model=default_model if default_model_type == "google" else None,
    default_ollama_model=default_model if default_model_type == "ollama" else None
)

# Text generation function (wrapper around model adapter)
async def generate_text(prompt: str, model_name: str = None, model_type: str = None, stream: bool = False) -> Union[str, Generator]:
    """Generate text using the model adapter."""
    response, confidence = await model_adapter.generate_response(
        prompt=prompt,
        model_type=model_type,
        model_name=model_name,
        stream=stream
    )
    return response

# Memory system for conversation history
class MemorySystem:
    def __init__(self, max_history: int = 10):
        self.max_history = max_history
        self.conversations: Dict[str, List[Dict[str, str]]] = {}
        self.current_conversation = "default"
        self.conversations[self.current_conversation] = []
    
    def add_message(self, role: str, content: str, conversation_id: str = None) -> None:
        """Add a message to the conversation history."""
        conv_id = conversation_id or self.current_conversation
        
        if conv_id not in self.conversations:
            self.conversations[conv_id] = []
        
        self.conversations[conv_id].append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })
        
        # Trim history if needed
        if len(self.conversations[conv_id]) > self.max_history:
            self.conversations[conv_id] = self.conversations[conv_id][-self.max_history:]
    
    def get_conversation(self, conversation_id: str = None) -> List[Dict[str, str]]:
        """Get the conversation history."""
        conv_id = conversation_id or self.current_conversation
        return self.conversations.get(conv_id, [])
    
    def clear_conversation(self, conversation_id: str = None) -> None:
        """Clear the conversation history."""
        conv_id = conversation_id or self.current_conversation
        if conv_id in self.conversations:
            self.conversations[conv_id] = []
    
    def set_current_conversation(self, conversation_id: str) -> None:
        """Set the current conversation."""
        self.current_conversation = conversation_id
        if conversation_id not in self.conversations:
            self.conversations[conversation_id] = []
    
    def get_all_conversations(self) -> Dict[str, List[Dict[str, str]]]:
        """Get all conversations."""
        return self.conversations
    
    def save_to_file(self, file_path: str = None) -> str:
        """Save conversations to a file."""
        if not file_path:
            file_path = Path(box1_exports["CONFIG_DIR"]) / "conversations.json"
        
        with open(file_path, "w") as f:
            json.dump(self.conversations, f, indent=2)
        
        return file_path
    
    def load_from_file(self, file_path: str = None) -> bool:
        """Load conversations from a file."""
        if not file_path:
            file_path = Path(box1_exports["CONFIG_DIR"]) / "conversations.json"
        
        if not os.path.exists(file_path):
            return False
        
        try:
            with open(file_path, "r") as f:
                self.conversations = json.load(f)
            return True
        except Exception as e:
            print(f"❌ Error loading conversations: {e}")
            return False

# Initialize memory system
memory = MemorySystem()
memory_file = Path(box1_exports["CONFIG_DIR"]) / "conversations.json"
if memory_file.exists():
    memory.load_from_file(str(memory_file))
    print(f"✅ Loaded conversation history from {memory_file}")
else:
    print("ℹ️ No existing conversation history found")

# Chat function that uses memory
async def chat(message: str, conversation_id: str = None, model_name: str = None, 
              model_type: str = None, stream: bool = False) -> Union[str, Generator]:
    """Chat with the model using conversation history."""
    conv_id = conversation_id or memory.current_conversation
    memory.add_message("user", message, conv_id)
    
    # Build prompt with conversation history
    conversation = memory.get_conversation(conv_id)
    prompt = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation])
    
    # Generate response
    response, confidence = await model_adapter.generate_response(
        prompt=prompt,
        model_type=model_type,
        model_name=model_name,
        stream=stream
    )
    
    if not stream:
        memory.add_message("assistant", response, conv_id)
    
    return response, confidence

# Test the model adapter
print("\n🧪 Testing model adapter with a simple query...")
test_response, test_confidence = await model_adapter.generate_response(
    "Hello, what can you do to help me with coding?",
    model_type=default_model_type,
    model_name=default_model
)
print(f"\nModel response (confidence: {test_confidence:.2f}):\n{test_response}")

# Export configuration for other boxes
box2_exports = {
    "default_model": default_model,
    "default_model_type": default_model_type,
    "google_models": google_models,
    "ollama_models": ollama_models,
    "google_ai_available": GOOGLE_AI_AVAILABLE,
    "ollama_available": OLLAMA_AVAILABLE,
    "memory_initialized": True
}

box2_exports_file = CONFIG_DIR / "box2_exports.json"
with open(box2_exports_file, "w") as f:
    json.dump(box2_exports, f, indent=2)

print("\n🎉 BOX 2 SETUP COMPLETED SUCCESSFULLY!")
print(f"🧠 Default Model Type: {default_model_type}")
print(f"🧠 Default Model: {default_model}")
print(f"🔢 Available Google Models: {len(google_models)}")
print(f"🔢 Available Ollama Models: {len(ollama_models)}")
print(f"💾 Memory System: Initialized")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🛠️ BOX 3: Tool Registry and Execution Framework - v1.0                                                   ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - Tool registration system                                                                               ║
# ║ - Tool execution pipeline                                                                                ║
# ║ - File system operations                                                                                 ║
# ║ - Code execution tools                                                                                   ║
# ║ - Research and deployment tools                                                                          ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🛠️ BOX 3: Initializing Tool Registry and Execution Framework...")

import os
import sys
import json
import time
import asyncio
import subprocess
import tempfile
import inspect
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable
from pydantic import BaseModel

# Load configuration from previous boxes
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box2_exports.json"), "r") as f:
        box2_exports = json.load(f)
    
    # Extract paths and configuration
    BASE_DIR = Path(box1_exports["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_exports["WORKSPACE_DIR"])
    CONFIG_DIR = Path(box1_exports["CONFIG_DIR"])
    TOOLS_DIR = Path(box1_exports["TOOLS_DIR"])
    IS_COLAB = box1_exports["IS_COLAB"]
    
    # Get model information
    default_model = box2_exports["default_model"]
    default_model_type = box2_exports["default_model_type"]
    GOOGLE_AI_AVAILABLE = box2_exports["google_ai_available"]
    OLLAMA_AVAILABLE = box2_exports["ollama_available"]
    
    print("✅ Loaded configuration from previous boxes")
except Exception as e:
    print(f"❌ Error loading previous box configurations: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if previous boxes haven't been run
    BASE_DIR = Path("./GoogleManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    CONFIG_DIR = BASE_DIR / "config"
    TOOLS_DIR = BASE_DIR / "tools"
    IS_COLAB = True
    default_model = "google/gemini-2.5-pro"
    default_model_type = "google"
    GOOGLE_AI_AVAILABLE = False
    OLLAMA_AVAILABLE = False

# Ensure workspace directory exists
WORKSPACE_DIR.mkdir(parents=True, exist_ok=True)
os.chdir(WORKSPACE_DIR)
print(f"📁 Working directory set to: {WORKSPACE_DIR}")

# Tool registry
TOOL_REGISTRY: Dict[str, Callable] = {}

# Tool registration decorator
def register_tool(name: str):
    """Decorator to register a tool in the registry."""
    def decorator(func):
        TOOL_REGISTRY[name] = func
        return func
    return decorator

# Tool call model
class ToolCall(BaseModel):
    tool_name: str
    tool_input: Dict[str, Any] = {}

# Task request model
class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None

# Tool execution function
async def call_tool(tool_call: ToolCall):
    """Execute a tool with the given input."""
    tool_name = tool_call.tool_name
    tool_input = tool_call.tool_input
    
    if tool_name not in TOOL_REGISTRY:
        return {"error": f"Tool {tool_name} not found"}
    
    try:
        result = TOOL_REGISTRY[tool_name](**tool_input)
        return result
    except Exception as e:
        return {"error": f"Error executing tool {tool_name}: {str(e)}"}

# List available tools
async def list_tools():
    """List all available tools with their descriptions and parameters."""
    tools_info = []
    
    for name, func in TOOL_REGISTRY.items():
        # Get function signature and docstring
        sig = inspect.signature(func)
        doc = inspect.getdoc(func) or ""
        
        # Extract parameters
        params = []
        for param_name, param in sig.parameters.items():
            param_info = {
                "name": param_name,
                "required": param.default == inspect.Parameter.empty,
                "type": str(param.annotation) if param.annotation != inspect.Parameter.empty else "Any"
            }
            params.append(param_info)
        
        tools_info.append({
            "name": name,
            "description": doc.split("\n")[0] if doc else "",
            "parameters": params
        })
    
    return {"tools": tools_info}

# ===== FILE SYSTEM TOOLS =====

@register_tool("write_file")
def write_file(file_path: str, content: str):
    """Write content to a file."""
    try:
        # Ensure the file path is within the workspace
        full_path = WORKSPACE_DIR / file_path
        
        # Create parent directories if they don't exist
        full_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Write the content
        with open(full_path, "w") as f:
            f.write(content)
        
        return {"success": True, "message": f"File written to {file_path}", "path": str(full_path)}
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("read_file")
def read_file(file_path: str):
    """Read content from a file."""
    try:
        # Ensure the file path is within the workspace
        full_path = WORKSPACE_DIR / file_path
        
        # Check if the file exists
        if not full_path.exists():
            return {"success": False, "error": f"File {file_path} not found"}
        
        # Read the content
        with open(full_path, "r") as f:
            content = f.read()
        
        return {"success": True, "content": content, "path": str(full_path)}
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("list_files")
def list_files(directory: str = ".", pattern: str = "*"):
    """List files in a directory."""
    try:
        # Ensure the directory path is within the workspace
        full_path = WORKSPACE_DIR / directory
        
        # Check if the directory exists
        if not full_path.exists():
            return {"success": False, "error": f"Directory {directory} not found"}
        
        # List files
        files = list(full_path.glob(pattern))
        file_list = [{
            "name": f.name,
            "path": str(f.relative_to(WORKSPACE_DIR)),
            "type": "directory" if f.is_dir() else "file",
            "size": f.stat().st_size if f.is_file() else None
        } for f in files]
        
        return {"success": True, "files": file_list, "directory": directory}
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("search_files")
def search_files(pattern: str, content_pattern: str = None, directory: str = "."):
    """Search for files by name pattern and optionally by content."""
    try:
        # Ensure the directory path is within the workspace
        full_path = WORKSPACE_DIR / directory
        
        # Check if the directory exists
        if not full_path.exists():
            return {"success": False, "error": f"Directory {directory} not found"}
        
        # Find files matching the pattern
        files = list(full_path.glob(f"**/{pattern}"))
        
        # If content pattern is provided, filter files by content
        if content_pattern:
            matching_files = []
            for file in files:
                if file.is_file():
                    try:
                        with open(file, "r") as f:
                            content = f.read()
                            if content_pattern in content:
                                matching_files.append(file)
                    except:
                        # Skip files that can't be read as text
                        pass
            files = matching_files
        
        # Format results
        file_list = [{
            "name": f.name,
            "path": str(f.relative_to(WORKSPACE_DIR)),
            "type": "directory" if f.is_dir() else "file",
            "size": f.stat().st_size if f.is_file() else None
        } for f in files]
        
        return {
            "success": True, 
            "files": file_list, 
            "count": len(file_list),
            "pattern": pattern,
            "content_pattern": content_pattern
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# ===== CODE EXECUTION TOOLS =====

@register_tool("execute_python")
def execute_python(code: str, timeout: int = 30):
    """Execute Python code and return the result."""
    try:
        # Create a temporary file
        with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as temp_file:
            temp_file.write(code.encode())
            temp_path = temp_file.name
        
        # Execute the code
        result = subprocess.run(
            [sys.executable, temp_path],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        
        # Clean up
        os.unlink(temp_path)
        
        return {
            "success": result.returncode == 0,
            "stdout": result.stdout,
            "stderr": result.stderr,
            "exit_code": result.returncode
        }
    except subprocess.TimeoutExpired:
        return {"success": False, "error": f"Execution timed out after {timeout} seconds"}
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("review_code")
def review_code(code: str, language: str = "python"):
    """Review code and provide suggestions for improvement."""
    try:
        # Import the model adapter from Box 2
        from __main__ import model_adapter
        
        # Prepare the prompt
        prompt = f"""Review the following {language} code and provide suggestions for improvement:

```{language}
{code}
```

Please analyze the code for:
1. Bugs and logical errors
2. Performance issues
3. Style and best practices
4. Security concerns
5. Potential improvements

For each issue, provide:
- A description of the issue
- The problematic code snippet
- A suggested fix

Finally, provide an overall assessment and a score from 1-10."""
        
        # Generate the review
        loop = asyncio.get_event_loop()
        review, confidence = loop.run_until_complete(
            model_adapter.generate_response(prompt)
        )
        
        return {
            "success": True,
            "review": review,
            "language": language,
            "confidence": confidence
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("generate_tests")
def generate_tests(code: str, language: str = "python"):
    """Generate unit tests for the given code."""
    try:
        # Import the model adapter from Box 2
        from __main__ import model_adapter
        
        # Prepare the prompt
        prompt = f"""Generate comprehensive unit tests for the following {language} code:

```{language}
{code}
```

Please create tests that:
1. Cover all functions and methods
2. Test edge cases and error conditions
3. Achieve high code coverage
4. Follow best practices for {language} testing

Return only the test code without explanations."""
        
        # Generate the tests
        loop = asyncio.get_event_loop()
        tests, confidence = loop.run_until_complete(
            model_adapter.generate_response(prompt)
        )
        
        return {
            "success": True,
            "tests": tests,
            "language": language,
            "confidence": confidence
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("install_package")
def install_package(package_name: str):
    """Install a Python package."""
    try:
        # Execute pip install
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", package_name],
            capture_output=True,
            text=True
        )
        
        return {
            "success": result.returncode == 0,
            "stdout": result.stdout,
            "stderr": result.stderr,
            "exit_code": result.returncode
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# ===== RESEARCH TOOLS =====

@register_tool("search_web")
def search_web(query: str, num_results: int = 5):
    """Search the web for information (simulated in this version)."""
    # In a real implementation, this would use a search API
    # For now, we'll just return a message
    return {
        "success": True,
        "message": f"Would search for: {query} (limited to {num_results} results)",
        "results": []
    }

@register_tool("summarize_text")
def summarize_text(text: str, max_length: int = 200):
    """Summarize a long text."""
    try:
        # Import the model adapter from Box 2
        from __main__ import model_adapter
        
        # Prepare the prompt
        prompt = f"""Summarize the following text in {max_length} words or less:

{text}

Summary:"""
        
        # Generate the summary
        loop = asyncio.get_event_loop()
        summary, confidence = loop.run_until_complete(
            model_adapter.generate_response(prompt)
        )
        
        return {
            "success": True,
            "summary": summary,
            "original_length": len(text),
            "summary_length": len(summary),
            "confidence": confidence
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# ===== DEPLOYMENT TOOLS =====

@register_tool("launch_site")
def launch_site(site_path: str, port: int = 8000):
    """Launch a website locally."""
    try:
        # Ensure the site path is within the workspace
        full_path = WORKSPACE_DIR / site_path
        
        # Create the directory if it doesn't exist
        full_path.mkdir(parents=True, exist_ok=True)
        
        # Create a simple index.html if it doesn't exist
        index_path = full_path / "index.html"
        if not index_path.exists():
            with open(index_path, "w") as f:
                f.write(f"""
                <!DOCTYPE html>
                <html>
                <head>
                    <title>Google Manus Site</title>
                    <style>
                        body {{ font-family: Arial, sans-serif; margin: 40px; }}
                        h1 {{ color: #4285F4; }}
                    </style>
                </head>
                <body>
                    <h1>Welcome to Google Manus Site</h1>
                    <p>This is a simple website launched by Google Manus System.</p>
                    <p>Current time: {time.strftime('%Y-%m-%d %H:%M:%S')}</p>
                </body>
                </html>""")
        
        # Create a simple server script
        server_path = full_path / "server.py"
        with open(server_path, "w") as f:
            f.write(f"""
import http.server
import socketserver
import os

PORT = {port}
DIRECTORY = '{full_path}'

class Handler(http.server.SimpleHTTPRequestHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, directory=DIRECTORY, **kwargs)
    
    def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        super().end_headers()

with socketserver.TCPServer(("", PORT), Handler) as httpd:
    print(f"Serving at port {{PORT}}")
    httpd.serve_forever()
            """)
        
        # Start the server in the background
        process = subprocess.Popen([sys.executable, str(server_path)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Wait a moment for the server to start
        time.sleep(2)
        
        # Check if the process is still running
        if process.poll() is not None:
            stdout, stderr = process.communicate()
            return {
                "success": False,
                "error": f"Server failed to start: {stderr.decode()}"
            }
        
        return {
            "success": True,
            "message": f"Site launched at http://localhost:{port}",
            "url": f"http://localhost:{port}",
            "site_path": str(full_path),
            "process_id": process.pid
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("setup_frp")
def setup_frp(local_port: int = 8000, custom_domain: str = None):
    """Setup FRP tunnel for public access."""
    try:
        # Download FRP if not exists
        frp_dir = BASE_DIR / "frp"
        if not frp_dir.exists():
            print("⏳ Downloading FRP...")
            os.makedirs(frp_dir, exist_ok=True)
            subprocess.run(["wget", "https://github.com/fatedier/frp/releases/download/v0.51.3/frp_0.51.3_linux_amd64.tar.gz", "-O", str(frp_dir / "frp.tar.gz")])
            subprocess.run(["tar", "-xzf", str(frp_dir / "frp.tar.gz"), "-C", str(frp_dir), "--strip-components=1"])
            os.remove(str(frp_dir / "frp.tar.gz"))
            print("✅ FRP downloaded and extracted")
        
        # Create FRP config
        domain = custom_domain or f"manus-{int(time.time())}.example.com"
        frp_config = f"""
[common]
server_addr = frp.example.com
server_port = 7000
token = your_token_here

[web]
type = http
local_port = {local_port}
custom_domains = {domain}
        """
        
        with open(frp_dir / "frpc.ini", "w") as f:
            f.write(frp_config)
        
        # In a real implementation, we would start FRP here
        # For now, we'll just return the configuration
        
        return {
            "success": True,
            "message": "FRP configuration created",
            "config_path": str(frp_dir / "frpc.ini"),
            "domain": domain,
            "local_port": local_port,
            "url": f"http://{domain}"
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# ===== AGENT TOOLS =====

@register_tool("action_agent")
def action_agent(task: str, context: str = None):
    """Execute a task using the agent."""
    try:
        # Import the model adapter from Box 2
        from __main__ import model_adapter
        
        # Prepare the prompt
        prompt = f"Task: {task}\n"
        if context:
            prompt += f"Context: {context}\n"
        
        prompt += "\nPlease help me complete this task. Provide a step-by-step approach and any code or commands needed."
        
        # Generate the response
        loop = asyncio.get_event_loop()
        response, confidence = loop.run_until_complete(
            model_adapter.generate_response(prompt)
        )
        
        return {
            "success": True,
            "task": task,
            "response": response,
            "confidence": confidence
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("get_agent_memory")
def get_agent_memory():
    """Get the agent's memory."""
    try:
        # Import the memory system from Box 2
        from __main__ import memory
        
        return {
            "success": True,
            "conversations": memory.get_all_conversations()
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

@register_tool("clear_agent_memory")
def clear_agent_memory(conversation_id: str = None):
    """Clear the agent's memory."""
    try:
        # Import the memory system from Box 2
        from __main__ import memory
        
        memory.clear_conversation(conversation_id)
        
        return {
            "success": True,
            "message": f"Memory cleared for conversation {conversation_id or 'default'}"
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# Print registered tools
print(f"✅ Registered {len(TOOL_REGISTRY)} tools")
for tool_name in TOOL_REGISTRY.keys():
    print(f"  - {tool_name}")

# Export configuration for other boxes
box3_exports = {
    "tools_registered": list(TOOL_REGISTRY.keys()),
    "workspace_dir": str(WORKSPACE_DIR)
}

box3_exports_file = CONFIG_DIR / "box3_exports.json"
with open(box3_exports_file, "w") as f:
    json.dump(box3_exports, f, indent=2)

print("\n🎉 BOX 3 SETUP COMPLETED SUCCESSFULLY!")
print(f"🛠️ Tools Registered: {len(TOOL_REGISTRY)}")
print(f"📁 Workspace Directory: {WORKSPACE_DIR}")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🖥️ BOX 4: User Interface and Interaction - v1.0                                                          ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - Jupyter widgets interface                                                                              ║
# ║ - Chat interface                                                                                         ║
# ║ - Task execution interface                                                                               ║
# ║ - Tool execution interface                                                                               ║
# ║ - Code editor interface                                                                                  ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🖥️ BOX 4: Initializing User Interface and Interaction...")

import os
import sys
import json
import time
import asyncio
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Load configuration from previous boxes
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box2_exports.json"), "r") as f:
        box2_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box3_exports.json"), "r") as f:
        box3_exports = json.load(f)
    
    # Extract configuration
    WORKSPACE_DIR = Path(box1_exports["WORKSPACE_DIR"])
    IS_COLAB = box1_exports["IS_COLAB"]
    config = box1_exports["config"]
    
    default_model = box2_exports["default_model"]
    default_model_type = box2_exports["default_model_type"]
    google_models = box2_exports["google_models"]
    ollama_models = box2_exports["ollama_models"]
    GOOGLE_AI_AVAILABLE = box2_exports["google_ai_available"]
    OLLAMA_AVAILABLE = box2_exports["ollama_available"]
    
    tools_registered = box3_exports["tools_registered"]
    
    print("✅ Loaded configuration from previous boxes")
except Exception as e:
    print(f"❌ Error loading previous box configurations: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if previous boxes haven't been run
    WORKSPACE_DIR = Path("./GoogleManusSystem/workspace")
    IS_COLAB = True
    config = {"human_review_enabled": True, "auto_approve_threshold": 0.8}
    
    default_model = "google/gemini-2.5-pro"
    default_model_type = "google"
    google_models = ["google/gemini-2.5-pro"]
    ollama_models = ["llama3"]
    GOOGLE_AI_AVAILABLE = False
    OLLAMA_AVAILABLE = False
    
    tools_registered = ["write_file", "read_file", "list_files", "execute_python", "action_agent"]

# Import functions from previous boxes
try:
    from __main__ import generate_text, chat, memory, call_tool, TOOL_REGISTRY, model_adapter
    print("✅ Imported functions from previous boxes")
except ImportError as e:
    print(f"❌ Error importing functions: {e}")
    print("⚠️ UI will have limited functionality")
    
    # Define placeholder functions
    async def generate_text(prompt, model_name=None, model_type=None, stream=False):
        return f"[PLACEHOLDER] Would generate text for: {prompt[:50]}..."
    
    async def chat(message, conversation_id=None, model_name=None, model_type=None, stream=False):
        return f"[PLACEHOLDER] Would chat about: {message[:50]}...", 0.5
    
    class MemorySystem:
        def get_conversation(self, conversation_id=None):
            return []
        
        def add_message(self, role, content, conversation_id=None):
            pass
        
        def clear_conversation(self, conversation_id=None):
            pass
    
    memory = MemorySystem()
    
    class ToolCall:
        def __init__(self, tool_name, tool_input):
            self.tool_name = tool_name
            self.tool_input = tool_input
    
    async def call_tool(tool_call):
        return {"error": "Tool execution not available"}
    
    TOOL_REGISTRY = {}
    for tool in tools_registered:
        TOOL_REGISTRY[tool] = lambda **kwargs: {"error": "Tool not implemented"}
    
    class ModelAdapter:
        def __init__(self):
            pass
        
        async def generate_response(self, prompt, model_type=None, model_name=None, stream=False):
            return f"[PLACEHOLDER] Would generate response for: {prompt[:50]}...", 0.5
        
        def list_available_models(self):
            return []
    
    model_adapter = ModelAdapter()

# ===== HELPER FUNCTIONS =====

def format_code(code, language="python"):
    """Format code with syntax highlighting."""
    return f"""<div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; margin: 10px 0;">
    <pre><code class="{language}">{code}</code></pre>
    </div>"""

def format_message(role, content):
    """Format a chat message."""
    if role == "user":
        color = "#e6f7ff"
        icon = "👤"
    else:
        color = "#f0f0f0"
        icon = "🤖"
    
    return f"""<div style="background-color: {color}; padding: 10px; border-radius: 5px; margin: 10px 0;">
    <strong>{icon} {role.capitalize()}</strong>
    <div style="margin-top: 5px;">{content}</div>
    </div>"""

def format_tool_result(result):
    """Format a tool execution result."""
    if isinstance(result, dict) and "error" in result:
        return f"""<div style="background-color: #ffe6e6; padding: 10px; border-radius: 5px; margin: 10px 0;">
        <strong>❌ Error</strong>
        <div style="margin-top: 5px;">{result['error']}</div>
        </div>"""
    
    if isinstance(result, dict) and "success" in result and not result["success"]:
        return f"""<div style="background-color: #ffe6e6; padding: 10px; border-radius: 5px; margin: 10px 0;">
        <strong>❌ Error</strong>
        <div style="margin-top: 5px;">{result.get('error', 'Unknown error')}</div>
        </div>"""
    
    # Format success result
    result_html = "<div style=\"background-color: #e6ffe6; padding: 10px; border-radius: 5px; margin: 10px 0;\">\n"
    result_html += "<strong>✅ Success</strong>\n"
    result_html += "<div style=\"margin-top: 5px;\">\n"
    
    if isinstance(result, dict):
        for key, value in result.items():
            if key in ["success", "error"]:
                continue
            
            if key == "stdout" and value:
                result_html += f"<strong>Output:</strong>\n"
                result_html += f"<pre>{value}</pre>\n"
            elif key == "stderr" and value:
                result_html += f"<strong>Errors:</strong>\n"
                result_html += f"<pre>{value}</pre>\n"
            elif key == "content" and value:
                result_html += f"<strong>Content:</strong>\n"
                result_html += f"<pre>{value}</pre>\n"
            elif key == "files" and value:
                result_html += f"<strong>Files:</strong>\n"
                result_html += "<ul>\n"
                for file in value:
                    result_html += f"<li>{file['name']} ({file['type']})</li>\n"
                result_html += "</ul>\n"
            elif key == "message" and value:
                result_html += f"<strong>Message:</strong> {value}\n"
            elif key == "url" and value:
                result_html += f"<strong>URL:</strong> <a href=\"{value}\" target=\"_blank\">{value}</a>\n"
            elif key == "response" and value:
                result_html += f"<strong>Response:</strong>\n"
                result_html += f"<div>{value}</div>\n"
            elif key == "review" and value:
                result_html += f"<strong>Review:</strong>\n"
                result_html += f"<div>{value}</div>\n"
            elif key == "tests" and value:
                result_html += f"<strong>Tests:</strong>\n"
                result_html += f"<pre>{value}</pre>\n"
            elif key == "summary" and value:
                result_html += f"<strong>Summary:</strong>\n"
                result_html += f"<div>{value}</div>\n"
    else:
        result_html += f"<pre>{result}</pre>\n"
    
    result_html += "</div>\n</div>"
    
    return result_html

# ===== UI COMPONENTS =====

# Header
header = widgets.HTML(
    value="<h1>🤖 Google Manus System</h1>"
)

# Model selection
model_type_dropdown = widgets.Dropdown(
    options=[
        ("Google", "google") if GOOGLE_AI_AVAILABLE else None,
        ("Ollama", "ollama") if OLLAMA_AVAILABLE else None,
        ("Fallback", "fallback") if not GOOGLE_AI_AVAILABLE and not OLLAMA_AVAILABLE else None
    ],
    value=default_model_type,
    description='Model Type:',
    disabled=not (GOOGLE_AI_AVAILABLE or OLLAMA_AVAILABLE),
    style={'description_width': '100px'}
)

# Filter out None values
model_type_dropdown.options = [opt for opt in model_type_dropdown.options if opt is not None]

# Google model selection
google_model_dropdown = widgets.Dropdown(
    options=google_models,
    value=default_model if default_model_type == "google" else (google_models[0] if google_models else None),
    description='Google Model:',
    disabled=not GOOGLE_AI_AVAILABLE or default_model_type != "google",
    style={'description_width': '100px'}
)

# Ollama model selection
ollama_model_dropdown = widgets.Dropdown(
    options=ollama_models,
    value=default_model if default_model_type == "ollama" else (ollama_models[0] if ollama_models else None),
    description='Ollama Model:',
    disabled=not OLLAMA_AVAILABLE or default_model_type != "ollama",
    style={'description_width': '100px'}
)

# Human review toggle
human_review_toggle = widgets.Checkbox(
    value=config.get("human_review_enabled", True),
    description='Enable Human Review',
    disabled=False,
    indent=False
)

# Tabs for different interfaces
tab = widgets.Tab()
tab_contents = []

# ===== CHAT INTERFACE =====

chat_output = widgets.Output()
chat_input = widgets.Text(
    value='',
    placeholder='Type a message...',
    description='',
    disabled=False
)
chat_button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='primary',
    tooltip='Send message',
    icon='paper-plane'
)
clear_chat_button = widgets.Button(
    description='Clear Chat',
    disabled=False,
    button_style='danger',
    tooltip='Clear chat history',
    icon='trash'
)

chat_controls = widgets.HBox([chat_input, chat_button, clear_chat_button])
chat_interface = widgets.VBox([chat_output, chat_controls])

# Chat event handlers
async def on_chat_button_clicked(b):
    message = chat_input.value
    if not message.strip():
        return
    
    chat_input.value = ''
    
    with chat_output:
        display(HTML(format_message("user", message)))
        
        # Add to memory
        memory.add_message("user", message)
        
        # Get the selected model
        model_type = model_type_dropdown.value
        model_name = None
        if model_type == "google":
            model_name = google_model_dropdown.value
        elif model_type == "ollama":
            model_name = ollama_model_dropdown.value
        
        # Generate response
        display(HTML("<p><em>Generating response...</em></p>"))
        response, confidence = await chat(message, model_name=model_name, model_type=model_type)
        
        # Check if human review is needed
        if human_review_toggle.value and confidence < config.get("auto_approve_threshold", 0.8):
            display(HTML(f"""<div style="background-color: #fff3cd; padding: 10px; border-radius: 5px; margin: 10px 0;">
            <strong>⚠️ Response pending human review (confidence: {confidence:.2f})</strong>
            <div style="margin-top: 5px;">The response will be displayed after human review.</div>
            </div>"""))
            
            # In a real implementation, this would queue the response for human review
            # For now, we'll just display it after a delay
            await asyncio.sleep(2)
            display(HTML("<p><em>Human review completed.</em></p>"))
        
        # Add to memory
        memory.add_message("assistant", response)
        
        # Display response
        display(HTML(format_message("assistant", response)))

def on_chat_input_submitted(sender):
    asyncio.create_task(on_chat_button_clicked(None))

def on_clear_chat_button_clicked(b):
    memory.clear_conversation()
    with chat_output:
        clear_output()
        display(HTML("<p>Chat history cleared.</p>"))

chat_button.on_click(lambda b: asyncio.create_task(on_chat_button_clicked(b)))
chat_input.on_submit(on_chat_input_submitted)
clear_chat_button.on_click(on_clear_chat_button_clicked)

# ===== TASK EXECUTION INTERFACE =====

task_input = widgets.Text(
    value='',
    placeholder='Enter a task...',
    description='Task:',
    disabled=False,
    style={'description_width': '100px'}
)
task_context = widgets.Textarea(
    value='',
    placeholder='Optional context...',
    description='Context:',
    disabled=False,
    style={'description_width': '100px'}
)
task_button = widgets.Button(
    description='Execute Task',
    disabled=False,
    button_style='success',
    tooltip='Execute the task',
    icon='play'
)
task_output = widgets.Output()

task_interface = widgets.VBox([task_input, task_context, task_button, task_output])

# Task event handlers
def on_task_button_clicked(b):
    task = task_input.value
    context = task_context.value
    
    if not task.strip():
        return
    
    with task_output:
        clear_output()
        display(HTML(f"<p><strong>Task:</strong> {task}</p>"))
        if context:
            display(HTML(f"<p><strong>Context:</strong> {context}</p>"))
        
        display(HTML("<p><em>Executing task...</em></p>"))
        
        # Call the action_agent tool
        if "action_agent" in TOOL_REGISTRY:
            result = TOOL_REGISTRY["action_agent"](task=task, context=context)
            display(HTML(format_tool_result(result)))
        else:
            display(HTML("<p>❌ Action agent tool not available.</p>"))

task_button.on_click(on_task_button_clicked)

# ===== TOOL EXECUTION INTERFACE =====

tool_dropdown = widgets.Dropdown(
    options=tools_registered,
    value=tools_registered[0] if tools_registered else None,
    description='Tool:',
    disabled=not tools_registered,
    style={'description_width': '100px'}
)
tool_input = widgets.Textarea(
    value='{}',
    placeholder='Enter tool input as JSON...',
    description='Input:',
    disabled=False,
    style={'description_width': '100px'}
)
tool_button = widgets.Button(
    description='Execute Tool',
    disabled=False,
    button_style='info',
    tooltip='Execute the selected tool',
    icon='wrench'
)
tool_output = widgets.Output()

tool_interface = widgets.VBox([tool_dropdown, tool_input, tool_button, tool_output])

# Tool event handlers
def on_tool_dropdown_change(change):
    tool_name = change['new']
    if tool_name == "write_file":
        tool_input.value = '{"file_path": "example.txt", "content": "Hello, world!"}'
    elif tool_name == "read_file":
        tool_input.value = '{"file_path": "example.txt"}'
    elif tool_name == "list_files":
        tool_input.value = '{"directory": ".", "pattern": "*"}'
    elif tool_name == "search_files":
        tool_input.value = '{"pattern": "*.py", "content_pattern": "import", "directory": "."}'
    elif tool_name == "execute_python":
        tool_input.value = '{"code": "print(\\'Hello, world!\\')\\nfor i in range(5):\\n    print(i)"}'
    elif tool_name == "review_code":
        tool_input.value = '{"code": "def fibonacci(n):\\n    if n <= 0:\\n        return 0\\n    elif n == 1:\\n        return 1\\n    else:\\n        return fibonacci(n-1) + fibonacci(n-2)", "language": "python"}'
    elif tool_name == "generate_tests":
        tool_input.value = '{"code": "def add(a, b):\\n    return a + b\\n\\ndef subtract(a, b):\\n    return a - b", "language": "python"}'
    elif tool_name == "action_agent":
        tool_input.value = '{"task": "Write a Python function to calculate Fibonacci numbers", "context": "The function should be efficient"}'
    elif tool_name == "summarize_text":
        tool_input.value = '{"text": "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.", "max_length": 50}'
    elif tool_name == "launch_site":
        tool_input.value = '{"site_path": "my_site", "port": 8000}'
    elif tool_name == "setup_frp":
        tool_input.value = '{"local_port": 8000, "custom_domain": "manus-example.com"}'
    else:
        tool_input.value = '{}'

def on_tool_button_clicked(b):
    tool_name = tool_dropdown.value
    input_json = tool_input.value
    
    try:
        input_dict = json.loads(input_json)
    except json.JSONDecodeError as e:
        with tool_output:
            clear_output()
            display(HTML(f"<p>❌ Invalid JSON: {str(e)}</p>"))
        return
    
    with tool_output:
        clear_output()
        display(HTML(f"<p><strong>Executing tool:</strong> {tool_name}</p>"))
        display(HTML(f"<p><strong>Input:</strong> {input_json}</p>"))
        
        # Execute the tool
        if tool_name in TOOL_REGISTRY:
            try:
                result = TOOL_REGISTRY[tool_name](**input_dict)
                display(HTML(format_tool_result(result)))
            except Exception as e:
                display(HTML(f"<p>❌ Error executing tool: {str(e)}</p>"))
        else:
            display(HTML("<p>❌ Tool not found.</p>"))

tool_dropdown.observe(on_tool_dropdown_change, names='value')
tool_button.on_click(on_tool_button_clicked)

# ===== CODE EDITOR INTERFACE =====

code_editor = widgets.Textarea(
    value='# Enter your Python code here\nprint("Hello, world!")',
    placeholder='Enter Python code...',
    description='Code:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': '100px'}
)
code_run_button = widgets.Button(
    description='Run Code',
    disabled=False,
    button_style='success',
    tooltip='Execute the code',
    icon='play'
)
code_review_button = widgets.Button(
    description='Review Code',
    disabled=False,
    button_style='info',
    tooltip='Review the code for improvements',
    icon='check'
)
code_test_button = widgets.Button(
    description='Generate Tests',
    disabled=False,
    button_style='warning',
    tooltip='Generate unit tests for the code',
    icon='vial'
)
code_output = widgets.Output()

code_buttons = widgets.HBox([code_run_button, code_review_button, code_test_button])
code_interface = widgets.VBox([code_editor, code_buttons, code_output])

# Code editor event handlers
def on_code_run_button_clicked(b):
    code = code_editor.value
    
    with code_output:
        clear_output()
        display(HTML("<p><strong>Executing code...</strong></p>"))
        
        # Execute the code
        if "execute_python" in TOOL_REGISTRY:
            result = TOOL_REGISTRY["execute_python"](code=code)
            display(HTML(format_tool_result(result)))
        else:
            display(HTML("<p>❌ Code execution tool not available.</p>"))

def on_code_review_button_clicked(b):
    code = code_editor.value
    
    with code_output:
        clear_output()
        display(HTML("<p><strong>Reviewing code...</strong></p>"))
        
        # Review the code
        if "review_code" in TOOL_REGISTRY:
            result = TOOL_REGISTRY["review_code"](code=code, language="python")
            display(HTML(format_tool_result(result)))
        else:
            display(HTML("<p>❌ Code review tool not available.</p>"))

def on_code_test_button_clicked(b):
    code = code_editor.value
    
    with code_output:
        clear_output()
        display(HTML("<p><strong>Generating tests...</strong></p>"))
        
        # Generate tests
        if "generate_tests" in TOOL_REGISTRY:
            result = TOOL_REGISTRY["generate_tests"](code=code, language="python")
            display(HTML(format_tool_result(result)))
        else:
            display(HTML("<p>❌ Test generation tool not available.</p>"))

code_run_button.on_click(on_code_run_button_clicked)
code_review_button.on_click(on_code_review_button_clicked)
code_test_button.on_click(on_code_test_button_clicked)

# ===== MODEL TYPE CHANGE HANDLER =====

def on_model_type_change(change):
    model_type = change['new']
    
    # Update model dropdowns
    if model_type == "google":
        google_model_dropdown.disabled = False
        ollama_model_dropdown.disabled = True
    elif model_type == "ollama":
        google_model_dropdown.disabled = True
        ollama_model_dropdown.disabled = False
    else:  # fallback
        google_model_dropdown.disabled = True
        ollama_model_dropdown.disabled = True

model_type_dropdown.observe(on_model_type_change, names='value')

# ===== ASSEMBLE THE UI =====

# Add tabs
tab_contents = [chat_interface, task_interface, tool_interface, code_interface]
tab.children = tab_contents
tab.set_title(0, "Chat")
tab.set_title(1, "Task Execution")
tab.set_title(2, "Tool Execution")
tab.set_title(3, "Code Editor")

# Model controls
model_controls = widgets.HBox([model_type_dropdown, google_model_dropdown, ollama_model_dropdown, human_review_toggle])

# Main UI
main_ui = widgets.VBox([header, model_controls, tab])

# Display the UI
display(main_ui)

# Initialize the chat interface
with chat_output:
    display(HTML("<p>Welcome to Google Manus System! How can I help you today?</p>"))

print("\n🎉 BOX 4 SETUP COMPLETED SUCCESSFULLY!")
print("🖥️ User Interface Initialized")
print("💬 Chat Interface Ready")
print("🎯 Task Execution Ready")
print("🛠️ Tool Execution Ready")
print("💻 Code Editor Ready")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 🌐 BOX 5: Web Deployment and External Services - v1.0                                                     ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - FastAPI server setup                                                                                   ║
# ║ - FRP tunnel integration                                                                                 ║
# ║ - Static file serving                                                                                    ║
# ║ - Web-based code execution environment                                                                   ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("🌐 BOX 5: Initializing Web Deployment and External Services...")

import os
import sys
import json
import time
import asyncio
import threading
import subprocess
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable

# Load configuration from previous boxes
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box2_exports.json"), "r") as f:
        box2_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box3_exports.json"), "r") as f:
        box3_exports = json.load(f)
    
    # Extract configuration
    BASE_DIR = Path(box1_exports["BASE_DIR"])
    WORKSPACE_DIR = Path(box1_exports["WORKSPACE_DIR"])
    SITES_DIR = Path(box1_exports["SITES_DIR"])
    HUMAN_REVIEW_DIR = Path(box1_exports["HUMAN_REVIEW_DIR"])
    IS_COLAB = box1_exports["IS_COLAB"]
    config = box1_exports["config"]
    
    default_model = box2_exports["default_model"]
    default_model_type = box2_exports["default_model_type"]
    GOOGLE_AI_AVAILABLE = box2_exports["google_ai_available"]
    OLLAMA_AVAILABLE = box2_exports["ollama_available"]
    
    tools_registered = box3_exports["tools_registered"]
    
    print("✅ Loaded configuration from previous boxes")
except Exception as e:
    print(f"❌ Error loading previous box configurations: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if previous boxes haven't been run
    BASE_DIR = Path("./GoogleManusSystem")
    WORKSPACE_DIR = BASE_DIR / "workspace"
    SITES_DIR = BASE_DIR / "sites"
    HUMAN_REVIEW_DIR = BASE_DIR / "human_review"
    IS_COLAB = True
    config = {"human_review_enabled": True, "auto_approve_threshold": 0.8}
    
    default_model = "google/gemini-2.5-pro"
    default_model_type = "google"
    GOOGLE_AI_AVAILABLE = False
    OLLAMA_AVAILABLE = False
    
    tools_registered = ["write_file", "read_file", "list_files", "execute_python", "action_agent"]

# Import functions from previous boxes
try:
    from __main__ import generate_text, chat, memory, call_tool, TOOL_REGISTRY, model_adapter
    print("✅ Imported functions from previous boxes")
except ImportError as e:
    print(f"❌ Error importing functions: {e}")
    print("⚠️ Web server will have limited functionality")
    
    # Define placeholder functions
    async def generate_text(prompt, model_name=None, model_type=None, stream=False):
        return f"[PLACEHOLDER] Would generate text for: {prompt[:50]}..."
    
    async def chat(message, conversation_id=None, model_name=None, model_type=None, stream=False):
        return f"[PLACEHOLDER] Would chat about: {message[:50]}...", 0.5
    
    class MemorySystem:
        def get_conversation(self, conversation_id=None):
            return []
        
        def add_message(self, role, content, conversation_id=None):
            pass
        
        def clear_conversation(self, conversation_id=None):
            pass
    
    memory = MemorySystem()
    
    class ToolCall:
        def __init__(self, tool_name, tool_input):
            self.tool_name = tool_name
            self.tool_input = tool_input
    
    async def call_tool(tool_call):
        return {"error": "Tool execution not available"}
    
    TOOL_REGISTRY = {}
    for tool in tools_registered:
        TOOL_REGISTRY[tool] = lambda **kwargs: {"error": "Tool not implemented"}
    
    class ModelAdapter:
        def __init__(self):
            pass
        
        async def generate_response(self, prompt, model_type=None, model_name=None, stream=False):
            return f"[PLACEHOLDER] Would generate response for: {prompt[:50]}...", 0.5
        
        def list_available_models(self):
            return []
    
    model_adapter = ModelAdapter()

# Create FastAPI server
from fastapi import FastAPI, Request, BackgroundTasks, HTTPException, Depends, Form, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from fastapi.staticfiles import StaticFiles
from fastapi.responses import JSONResponse, HTMLResponse, FileResponse, StreamingResponse
from pydantic import BaseModel
import uvicorn

app = FastAPI(title="Google Manus API", description="API for Google Manus System", version="1.0.0")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# API models
class ChatRequest(BaseModel):
    message: str
    conversation_id: Optional[str] = None
    model_name: Optional[str] = None
    model_type: Optional[str] = None
    user_id: Optional[str] = None

class ToolRequest(BaseModel):
    tool_name: str
    tool_input: Dict[str, Any] = {}

class TaskRequest(BaseModel):
    task: str
    context: Optional[str] = None
    model_name: Optional[str] = None
    model_type: Optional[str] = None

class ReviewSubmission(BaseModel):
    review_id: str
    approved: bool
    modified_response: Optional[str] = None

# Message queue for human review
class MessageQueue:
    def __init__(self):
        self.messages = {}
        self.next_id = 1
        
        # Load existing messages if available
        queue_file = HUMAN_REVIEW_DIR / "message_queue.json"
        if queue_file.exists():
            try:
                with open(queue_file, "r") as f:
                    data = json.load(f)
                    self.messages = data.get("messages", {})
                    self.next_id = data.get("next_id", 1)
            except Exception as e:
                print(f"Error loading message queue: {e}")
    
    def add_message(self, user_id, message, ai_response, confidence, conversation_id=None):
        """Add a message to the queue."""
        review_id = str(self.next_id)
        self.next_id += 1
        
        self.messages[review_id] = {
            "user_id": user_id,
            "message": message,
            "ai_response": ai_response,
            "confidence": confidence,
            "conversation_id": conversation_id,
            "status": "pending",
            "created_at": time.time(),
            "final_response": None
        }
        
        # Save to file
        self._save_to_file()
        
        return review_id
    
    def get_message(self, review_id):
        """Get a message by ID."""
        return self.messages.get(review_id)
    
    def update_message(self, review_id, status, final_response=None):
        """Update a message's status and final response."""
        if review_id in self.messages:
            self.messages[review_id]["status"] = status
            if final_response:
                self.messages[review_id]["final_response"] = final_response
            self.messages[review_id]["updated_at"] = time.time()
            
            # Save to file
            self._save_to_file()
            
            return True
        return False
    
    def get_pending_messages(self, limit=10):
        """Get pending messages awaiting review."""
        pending = [
            {"review_id": review_id, **message}
            for review_id, message in self.messages.items()
            if message["status"] == "pending"
        ]
        
        # Sort by creation time (oldest first)
        pending.sort(key=lambda x: x["created_at"])
        
        return pending[:limit]
    
    def _save_to_file(self):
        """Save the message queue to a file."""
        queue_file = HUMAN_REVIEW_DIR / "message_queue.json"
        with open(queue_file, "w") as f:
            json.dump({"messages": self.messages, "next_id": self.next_id}, f, indent=2)

# Initialize message queue
message_queue = MessageQueue()

# Chat router for human-in-the-loop
class ChatRouter:
    def __init__(self, model_adapter, message_queue, human_review_enabled=True, auto_approve_threshold=0.8):
        self.model_adapter = model_adapter
        self.message_queue = message_queue
        self.human_review_enabled = human_review_enabled
        self.auto_approve_threshold = auto_approve_threshold
    
    async def process_message(self, user_id, message, conversation_id=None, model_name=None, model_type=None):
        """Process an incoming user message."""
        # Generate AI response
        ai_response, confidence = await self.model_adapter.generate_response(
            prompt=message,
            model_type=model_type,
            model_name=model_name
        )
        
        # Add user message to memory
        memory.add_message("user", message, conversation_id)
        
        if not self.human_review_enabled or confidence >= self.auto_approve_threshold:
            # Auto-approve high-confidence responses
            memory.add_message("assistant", ai_response, conversation_id)
            return {"response": ai_response, "confidence": confidence, "reviewed": False}
        
        # Queue for human review
        review_id = self.message_queue.add_message(
            user_id=user_id,
            message=message,
            ai_response=ai_response,
            confidence=confidence,
            conversation_id=conversation_id
        )
        
        # Return a temporary response
        return {
            "status": "pending_review",
            "review_id": review_id,
            "confidence": confidence,
            "message": "Your message is being reviewed by a human operator."
        }
    
    async def get_pending_reviews(self, limit=10):
        """Get pending messages awaiting human review."""
        return self.message_queue.get_pending_messages(limit)
    
    async def submit_human_review(self, review_id, approved, modified_response=None):
        """Submit human review for a message."""
        message = self.message_queue.get_message(review_id)
        if not message:
            return {"error": "Message not found"}
        
        final_response = modified_response if modified_response else message["ai_response"]
        
        # Update message status
        self.message_queue.update_message(
            review_id=review_id,
            status="approved" if approved else "rejected",
            final_response=final_response
        )
        
        # If approved, add to memory
        if approved:
            memory.add_message("assistant", final_response, message["conversation_id"])
        
        return {"status": "success", "approved": approved}

# Initialize chat router
chat_router = ChatRouter(
    model_adapter=model_adapter,
    message_queue=message_queue,
    human_review_enabled=config.get("human_review_enabled", True),
    auto_approve_threshold=config.get("auto_approve_threshold", 0.8)
)

# API routes
@app.get("/", response_class=HTMLResponse)
async def root():
    return """
    <html>
        <head>
            <title>Google Manus API</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }
                h1 { color: #4285F4; }
                h2 { color: #34A853; margin-top: 30px; }
                pre { background-color: #f5f5f5; padding: 10px; border-radius: 5px; }
                .endpoint { margin-bottom: 20px; }
            </style>
        </head>
        <body>
            <h1>🤖 Google Manus API</h1>
            <p>Welcome to the Google Manus API. Use the following endpoints to interact with the system:</p>
            
            <div class="endpoint">
                <h2>📋 API Documentation</h2>
                <p>View the full API documentation at <a href="/docs">/docs</a></p>
            </div>
            
            <div class="endpoint">
                <h2>💬 Chat</h2>
                <p>Send a message to the model:</p>
                <pre>POST /api/chat
{
    "message": "Hello, how are you?",
    "conversation_id": "optional-id",
    "model_name": "optional-model-name",
    "model_type": "google|ollama"
}</pre>
            </div>
            
            <div class="endpoint">
                <h2>👤 Human-in-the-Loop Chat</h2>
                <p>Send a message for human review:</p>
                <pre>POST /api/chat/human-review
{
    "message": "Hello, how are you?",
    "conversation_id": "optional-id",
    "model_name": "optional-model-name",
    "model_type": "google|ollama",
    "user_id": "optional-user-id"
}</pre>
            </div>
            
            <div class="endpoint">
                <h2>🛠️ Tool Execution</h2>
                <p>Execute a tool:</p>
                <pre>POST /api/tool
{
    "tool_name": "write_file",
    "tool_input": {
        "file_path": "example.txt",
        "content": "Hello, world!"
    }
}</pre>
            </div>
            
            <div class="endpoint">
                <h2>🎯 Task Execution</h2>
                <p>Execute a task:</p>
                <pre>POST /api/task
{
    "task": "Write a Python function to calculate Fibonacci numbers",
    "context": "The function should be efficient",
    "model_name": "optional-model-name",
    "model_type": "google|ollama"
}</pre>
            </div>
            
            <div class="endpoint">
                <h2>🧠 Models</h2>
                <p>List available models:</p>
                <pre>GET /api/models</pre>
            </div>
            
            <div class="endpoint">
                <h2>🛠️ Tools</h2>
                <p>List available tools:</p>
                <pre>GET /api/tools</pre>
            </div>
            
            <div class="endpoint">
                <h2>👤 Human Review</h2>
                <p>Get pending reviews:</p>
                <pre>GET /api/reviews/pending</pre>
                <p>Submit a review:</p>
                <pre>POST /api/reviews/submit
{
    "review_id": "1",
    "approved": true,
    "modified_response": "Optional modified response"
}</pre>
            </div>
        </body>
    </html>
    """

@app.get("/api/models")
async def get_models():
    """List available models."""
    models = model_adapter.list_available_models()
    return {
        "models": models,
        "default_model": default_model,
        "default_model_type": default_model_type
    }

@app.get("/api/tools")
async def get_tools():
    """List available tools."""
    return {"tools": tools_registered}

@app.post("/api/chat")
async def api_chat(request: ChatRequest):
    """Chat with the model."""
    try:
        response, confidence = await chat(
            message=request.message,
            conversation_id=request.conversation_id,
            model_name=request.model_name,
            model_type=request.model_type
        )
        return {"response": response, "confidence": confidence}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/api/chat/human-review")
async def api_chat_with_review(request: ChatRequest):
    """Chat with human review."""
    try:
        result = await chat_router.process_message(
            user_id=request.user_id or "anonymous",
            message=request.message,
            conversation_id=request.conversation_id,
            model_name=request.model_name,
            model_type=request.model_type
        )
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/api/tool")
async def api_tool(request: ToolRequest):
    """Execute a tool."""
    try:
        if request.tool_name not in TOOL_REGISTRY:
            raise HTTPException(status_code=404, detail=f"Tool {request.tool_name} not found")
        
        result = TOOL_REGISTRY[request.tool_name](**request.tool_input)
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/api/task")
async def api_task(request: TaskRequest):
    """Execute a task."""
    try:
        if "action_agent" not in TOOL_REGISTRY:
            raise HTTPException(status_code=404, detail="Action agent tool not found")
        
        result = TOOL_REGISTRY["action_agent"](task=request.task, context=request.context)
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/api/reviews/pending")
async def get_pending_reviews(limit: int = 10):
    """Get pending messages awaiting human review."""
    try:
        pending_messages = await chat_router.get_pending_reviews(limit)
        return {"reviews": pending_messages}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/api/reviews/submit")
async def submit_review(submission: ReviewSubmission):
    """Submit a human review for a message."""
    try:
        result = await chat_router.submit_human_review(
            review_id=submission.review_id,
            approved=submission.approved,
            modified_response=submission.modified_response
        )
        
        if "error" in result:
            return {"status": "error", "error": result["error"]}
        
        return {"status": "success", "approved": submission.approved}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Mount static files for sites
app.mount("/site", StaticFiles(directory=str(SITES_DIR)), name="sites")

# Create a simple site launcher
@app.get("/site/launch/{site_name}")
async def launch_site(site_name: str):
    site_path = SITES_DIR / site_name
    
    if not site_path.exists():
        site_path.mkdir(parents=True, exist_ok=True)
        
        # Create a simple index.html
        index_path = site_path / "index.html"
        with open(index_path, "w") as f:
            f.write(f"""
            <!DOCTYPE html>
            <html>
            <head>
                <title>{site_name} - Google Manus Site</title>
                <style>
                    body {{ font-family: Arial, sans-serif; margin: 40px; }}
                    h1 {{ color: #4285F4; }}
                </style>
            </head>
            <body>
                <h1>Welcome to {site_name}</h1>
                <p>This is a simple website launched by Google Manus System.</p>
                <p>Current time: {time.strftime('%Y-%m-%d %H:%M:%S')}</p>
            </body>
            </html>
            """)
    
    # Return the index.html file
    return FileResponse(site_path / "index.html")

# Human review interface
@app.get("/human-review", response_class=HTMLResponse)
async def human_review_interface():
    """Web interface for human review."""
    return """
    <html>
        <head>
            <title>Human Review Interface</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }
                h1 { color: #4285F4; }
                h2 { color: #34A853; margin-top: 30px; }
                .review-item { border: 1px solid #ddd; padding: 15px; margin-bottom: 20px; border-radius: 5px; }
                .user-message { background-color: #e6f7ff; padding: 10px; border-radius: 5px; margin: 10px 0; }
                .ai-response { background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin: 10px 0; }
                .confidence { color: #666; }
                .actions { margin-top: 15px; }
                button { padding: 8px 16px; margin-right: 10px; border-radius: 4px; cursor: pointer; }
                .approve { background-color: #34A853; color: white; border: none; }
                .reject { background-color: #EA4335; color: white; border: none; }
                .refresh { background-color: #4285F4; color: white; border: none; }
                textarea { width: 100%; height: 100px; margin-top: 10px; padding: 8px; border-radius: 4px; border: 1px solid #ddd; }
                #pending-reviews { margin-top: 20px; }
            </style>
        </head>
        <body>
            <h1>🧠 Human Review Interface</h1>
            <p>Review and approve AI-generated responses before they are sent to users.</p>
            
            <button class="refresh" onclick="loadPendingReviews()">Refresh Pending Reviews</button>
            
            <div id="pending-reviews">
                <p>Loading pending reviews...</p>
            </div>
            
            <script>
                // Load pending reviews
                function loadPendingReviews() {
                    fetch('/api/reviews/pending')
                        .then(response => response.json())
                        .then(data => {
                            const pendingReviews = document.getElementById('pending-reviews');
                            
                            if (data.reviews.length === 0) {
                                pendingReviews.innerHTML = '<p>No pending reviews.</p>';
                                return;
                            }
                            
                            let html = '';
                            data.reviews.forEach(review => {
                                html += `
                                    <div class="review-item" id="review-${review.review_id}">
                                        <h3>Review #${review.review_id}</h3>
                                        <p><strong>User:</strong> ${review.user_id}</p>
                                        <p><strong>Created:</strong> ${new Date(review.created_at * 1000).toLocaleString()}</p>
                                        <p class="confidence"><strong>Confidence:</strong> ${review.confidence.toFixed(2)}</p>
                                        
                                        <div class="user-message">
                                            <strong>User Message:</strong>
                                            <div>${review.message}</div>
                                        </div>
                                        
                                        <div class="ai-response">
                                            <strong>AI Response:</strong>
                                            <div>${review.ai_response}</div>
                                        </div>
                                        
                                        <textarea id="modified-${review.review_id}" placeholder="Edit the response if needed...">${review.ai_response}</textarea>
                                        
                                        <div class="actions">
                                            <button class="approve" onclick="approveReview('${review.review_id}')">Approve</button>
                                            <button class="reject" onclick="rejectReview('${review.review_id}')">Reject</button>
                                        </div>
                                    </div>
                                `;
                            });
                            
                            pendingReviews.innerHTML = html;
                        })
                        .catch(error => {
                            console.error('Error loading reviews:', error);
                            document.getElementById('pending-reviews').innerHTML = `<p>Error loading reviews: ${error.message}</p>`;
                        });
                }
                
                // Approve a review
                function approveReview(reviewId) {
                    const modifiedResponse = document.getElementById(`modified-${reviewId}`).value;
                    
                    fetch('/api/reviews/submit', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json'
                        },
                        body: JSON.stringify({
                            review_id: reviewId,
                            approved: true,
                            modified_response: modifiedResponse
                        })
                    })
                    .then(response => response.json())
                    .then(data => {
                        if (data.status === 'success') {
                            document.getElementById(`review-${reviewId}`).innerHTML = `<p>Review #${reviewId} approved successfully.</p>`;
                            setTimeout(() => loadPendingReviews(), 1000);
                        } else {
                            alert(`Error: ${data.error || 'Unknown error'}`)
                        }
                    })
                    .catch(error => {
                        console.error('Error approving review:', error);
                        alert(`Error approving review: ${error.message}`);
                    });
                }
                
                // Reject a review
                function rejectReview(reviewId) {
                    fetch('/api/reviews/submit', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json'
                        },
                        body: JSON.stringify({
                            review_id: reviewId,
                            approved: false
                        })
                    })
                    .then(response => response.json())
                    .then(data => {
                        if (data.status === 'success') {
                            document.getElementById(`review-${reviewId}`).innerHTML = `<p>Review #${reviewId} rejected successfully.</p>`;
                            setTimeout(() => loadPendingReviews(), 1000);
                        } else {
                            alert(`Error: ${data.error || 'Unknown error'}`)
                        }
                    })
                    .catch(error => {
                        console.error('Error rejecting review:', error);
                        alert(`Error rejecting review: ${error.message}`);
                    });
                }
                
                // Load reviews on page load
                document.addEventListener('DOMContentLoaded', loadPendingReviews);
            </script>
        </body>
    </html>
    """

# Setup FRP tunnel
def setup_frp(port: int = 8000):
    """Setup FRP tunnel for public access."""
    try:
        # Download FRP if not exists
        frp_dir = BASE_DIR / "frp"
        if not frp_dir.exists():
            print("⏳ Downloading FRP...")
            os.makedirs(frp_dir, exist_ok=True)
            subprocess.run(["wget", "https://github.com/fatedier/frp/releases/download/v0.51.3/frp_0.51.3_linux_amd64.tar.gz", "-O", str(frp_dir / "frp.tar.gz")])
            subprocess.run(["tar", "-xzf", str(frp_dir / "frp.tar.gz"), "-C", str(frp_dir), "--strip-components=1"])
            os.remove(str(frp_dir / "frp.tar.gz"))
            print("✅ FRP downloaded and extracted")
        
        # Create FRP config
        frp_config = f"""
[common]
server_addr = frp.example.com
server_port = 7000
token = your_token_here

[web]
type = http
local_port = {port}
custom_domains = your-domain.example.com
        """
        
        with open(frp_dir / "frpc.ini", "w") as f:
            f.write(frp_config)
        
        print("⚠️ FRP configuration created, but not started")
        print("ℹ️ To use FRP, you need to configure a real FRP server")
        print(f"ℹ️ Edit the configuration file at {frp_dir / 'frpc.ini'}")
        print(f"ℹ️ Then run: {frp_dir / 'frpc'} -c {frp_dir / 'frpc.ini'}")
        
        return {"success": True, "message": "FRP configuration created"}
    except Exception as e:
        print(f"❌ Error setting up FRP: {e}")
        return {"success": False, "error": str(e)}

# Start the server in a separate thread
def start_server():
    """Start the FastAPI server."""
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Setup FRP
frp_result = setup_frp()

# Start the server in a background thread
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True  # Allow the thread to be terminated when the notebook is closed
server_thread.start()

# Export configuration for other boxes
box5_exports = {
    "server_started": True,
    "server_port": 8000,
    "frp_configured": frp_result["success"],
    "human_review_enabled": config.get("human_review_enabled", True)
}

box5_exports_file = CONFIG_DIR / "box5_exports.json"
with open(box5_exports_file, "w") as f:
    json.dump(box5_exports, f, indent=2)

print("\n🎉 BOX 5 SETUP COMPLETED SUCCESSFULLY!")
print("🌐 Web Server Started at http://localhost:8000")
print("📚 API Documentation available at http://localhost:8000/docs")
print("👤 Human Review Interface available at http://localhost:8000/human-review")
print("🚀 Site Launcher available at http://localhost:8000/site/launch/{site_name}")
print("⚙️ FRP Configuration Created (Manual Setup Required)")

# ╔══════════════════════════════════════════════════════════════════════════════════════════════════════════╗
# ║ 👤 BOX 6: Human-in-the-Loop System - v1.0                                                                ║
# ║                                                                                                          ║
# ╟────────────────────────────────────── CORE FEATURES ─────────────────────────────────────────────────────╢
# ║ - Human review interface                                                                                 ║
# ║ - Message queue management                                                                               ║
# ║ - Response approval workflow                                                                             ║
# ║ - Confidence threshold configuration                                                                     ║
# ╚══════════════════════════════════════════════════════════════════════════════════════════════════════════╝

In [ ]:
print("👤 BOX 6: Initializing Human-in-the-Loop System...")

import os
import sys
import json
import time
import asyncio
from pathlib import Path
from typing import Dict, List, Any, Optional, Union, Callable
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Load configuration from previous boxes
try:
    with open(Path("./GoogleManusSystem/config/box1_exports.json"), "r") as f:
        box1_exports = json.load(f)
    
    with open(Path("./GoogleManusSystem/config/box5_exports.json"), "r") as f:
        box5_exports = json.load(f)
    
    # Extract configuration
    HUMAN_REVIEW_DIR = Path(box1_exports["HUMAN_REVIEW_DIR"])
    config = box1_exports["config"]
    
    human_review_enabled = config.get("human_review_enabled", True)
    auto_approve_threshold = config.get("auto_approve_threshold", 0.8)
    server_started = box5_exports.get("server_started", False)
    server_port = box5_exports.get("server_port", 8000)
    
    print("✅ Loaded configuration from previous boxes")
except Exception as e:
    print(f"❌ Error loading previous box configurations: {e}")
    print("⚠️ Using default configuration")
    
    # Default configuration if previous boxes haven't been run
    HUMAN_REVIEW_DIR = Path("./GoogleManusSystem/human_review")
    human_review_enabled = True
    auto_approve_threshold = 0.8
    server_started = False
    server_port = 8000

# Import functions from previous boxes
try:
    from __main__ import message_queue, chat_router
    print("✅ Imported functions from previous boxes")
except ImportError as e:
    print(f"❌ Error importing functions: {e}")
    print("⚠️ Human review interface will have limited functionality")
    
    # Define placeholder classes
    class MessageQueue:
        def __init__(self):
            self.messages = {}
            self.next_id = 1
        
        def add_message(self, user_id, message, ai_response, confidence, conversation_id=None):
            review_id = str(self.next_id)
            self.next_id += 1
            
            self.messages[review_id] = {
                "user_id": user_id,
                "message": message,
                "ai_response": ai_response,
                "confidence": confidence,
                "conversation_id": conversation_id,
                "status": "pending",
                "created_at": time.time(),
                "final_response": None
            }
            
            return review_id
        
        def get_message(self, review_id):
            return self.messages.get(review_id)
        
        def update_message(self, review_id, status, final_response=None):
            if review_id in self.messages:
                self.messages[review_id]["status"] = status
                if final_response:
                    self.messages[review_id]["final_response"] = final_response
                self.messages[review_id]["updated_at"] = time.time()
                return True
            return False
        
        def get_pending_messages(self, limit=10):
            pending = [
                {"review_id": review_id, **message}
                for review_id, message in self.messages.items()
                if message["status"] == "pending"
            ]
            
            pending.sort(key=lambda x: x["created_at"])
            
            return pending[:limit]
    
    message_queue = MessageQueue()
    
    class ChatRouter:
        def __init__(self, message_queue, human_review_enabled=True, auto_approve_threshold=0.8):
            self.message_queue = message_queue
            self.human_review_enabled = human_review_enabled
            self.auto_approve_threshold = auto_approve_threshold
        
        async def get_pending_reviews(self, limit=10):
            return self.message_queue.get_pending_messages(limit)
        
        async def submit_human_review(self, review_id, approved, modified_response=None):
            message = self.message_queue.get_message(review_id)
            if not message:
                return {"error": "Message not found"}
            
            final_response = modified_response if modified_response else message["ai_response"]
            
            self.message_queue.update_message(
                review_id=review_id,
                status="approved" if approved else "rejected",
                final_response=final_response
            )
            
            return {"status": "success", "approved": approved}
    
    chat_router = ChatRouter(message_queue, human_review_enabled, auto_approve_threshold)

# Create a Jupyter widget interface for human review
def create_human_review_interface():
    """Create a GUI for human review of AI responses."""
    # Header
    header = widgets.HTML(
        value="<h2>🧠 Human-in-the-Loop Review Interface</h2>"
    )
    
    # Settings
    human_review_toggle = widgets.Checkbox(
        value=human_review_enabled,
        description='Enable Human Review',
        disabled=False,
        indent=False
    )
    
    threshold_slider = widgets.FloatSlider(
        value=auto_approve_threshold,
        min=0.0,
        max=1.0,
        step=0.05,
        description='Auto-approve Threshold:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.2f',
        style={'description_width': '180px'}
    )
    
    settings_box = widgets.VBox([human_review_toggle, threshold_slider])
    
    # Pending messages list
    pending_list = widgets.Select(
        options=[],
        description='Pending:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='150px')
    )
    
    # Message details
    user_message = widgets.HTML(
        value="<p><strong>User Message:</strong> Select a message to review</p>"
    )
    
    ai_response = widgets.Textarea(
        value='',
        placeholder='AI response will appear here. You can edit it.',
        description='AI Response:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='150px')
    )
    
    confidence = widgets.HTML(
        value="<p><strong>Confidence:</strong> N/A</p>"
    )
    
    # Action buttons
    approve_button = widgets.Button(
        description='Approve',
        disabled=True,
        button_style='success',
        tooltip='Approve this response',
        icon='check'
    )
    
    reject_button = widgets.Button(
        description='Reject',
        disabled=True,
        button_style='danger',
        tooltip='Reject this response',
        icon='times'
    )
    
    refresh_button = widgets.Button(
        description='Refresh',
        disabled=False,
        button_style='info',
        tooltip='Refresh the pending list',
        icon='refresh'
    )
    
    status = widgets.HTML(
        value="<p>Ready to review messages.</p>"
    )
    
    # Web interface link
    web_interface_link = widgets.HTML(
        value=f"<p>Web interface available at: <a href='http://localhost:{server_port}/human-review' target='_blank'>http://localhost:{server_port}/human-review</a></p>" if server_started else "<p>Web server not started. Run Box 5 to enable web interface.</p>"
    )
    
    # Output area for debugging
    output = widgets.Output()
    
    # Layout
    buttons = widgets.HBox([approve_button, reject_button, refresh_button])
    details = widgets.VBox([user_message, ai_response, confidence, buttons, status])
    main_ui = widgets.VBox([header, settings_box, web_interface_link, pending_list, details, output])
    
    # Current review ID
    current_review_id = None
    
    # Event handlers
    async def refresh_pending_list(b=None):
        with output:
            clear_output()
            print("Refreshing pending messages...")
        
        pending_messages = await chat_router.get_pending_reviews()
        
        options = []
        for message in pending_messages:
            user_msg = message["message"]
            if len(user_msg) > 30:
                user_msg = user_msg[:30] + "..."
            options.append((f"{message['review_id']}: {user_msg}", message["review_id"]))
        
        pending_list.options = options
        
        with output:
            print(f"Found {len(pending_messages)} pending messages.")
    
    async def on_pending_selection(change):
        nonlocal current_review_id
        
        if not change["new"]:
            return
        
        review_id = change["new"]
        current_review_id = review_id
        
        with output:
            clear_output()
            print(f"Loading message {review_id}...")
        
        message = message_queue.get_message(review_id)
        
        if not message:
            with output:
                print(f"Message {review_id} not found.")
            return
        
        user_message.value = f"<p><strong>User Message:</strong> {message['message']}</p>"
        ai_response.value = message["ai_response"]
        confidence.value = f"<p><strong>Confidence:</strong> {message['confidence']:.2f}</p>"
        
        approve_button.disabled = False
        reject_button.disabled = False
        
        with output:
            print(f"Loaded message {review_id}.")
    
    async def on_approve_clicked(b):
        if not current_review_id:
            return
        
        with output:
            clear_output()
            print(f"Approving message {current_review_id}...")
        
        result = await chat_router.submit_human_review(
            review_id=current_review_id,
            approved=True,
            modified_response=ai_response.value
        )
        
        status.value = f"<p>✅ Approved message {current_review_id}.</p>"
        approve_button.disabled = True
        reject_button.disabled = True
        
        await refresh_pending_list()
    
    async def on_reject_clicked(b):
        if not current_review_id:
            return
        
        with output:
            clear_output()
            print(f"Rejecting message {current_review_id}...")
        
        result = await chat_router.submit_human_review(
            review_id=current_review_id,
            approved=False
        )
        
        status.value = f"<p>❌ Rejected message {current_review_id}.</p>"
        approve_button.disabled = True
        reject_button.disabled = True
        
        await refresh_pending_list()
    
    def on_human_review_toggle(change):
        new_value = change["new"]
        chat_router.human_review_enabled = new_value
        
        # Update config
        config["human_review_enabled"] = new_value
        config_file = Path("./GoogleManusSystem/config/system_config.json")
        with open(config_file, "w") as f:
            json.dump(config, f, indent=2)
        
        with output:
            clear_output()
            print(f"Human review {'enabled' if new_value else 'disabled'}.")
    
    def on_threshold_change(change):
        new_value = change["new"]
        chat_router.auto_approve_threshold = new_value
        
        # Update config
        config["auto_approve_threshold"] = new_value
        config_file = Path("./GoogleManusSystem/config/system_config.json")
        with open(config_file, "w") as f:
            json.dump(config, f, indent=2)
        
        with output:
            clear_output()
            print(f"Auto-approve threshold set to {new_value:.2f}.")
    
    # Connect event handlers
    pending_list.observe(lambda c: asyncio.create_task(on_pending_selection(c)), names='value')
    approve_button.on_click(lambda b: asyncio.create_task(on_approve_clicked(b)))
    reject_button.on_click(lambda b: asyncio.create_task(on_reject_clicked(b)))
    refresh_button.on_click(lambda b: asyncio.create_task(refresh_pending_list(b)))
    human_review_toggle.observe(on_human_review_toggle, names='value')
    threshold_slider.observe(on_threshold_change, names='value')
    
    # Initial refresh
    asyncio.create_task(refresh_pending_list())
    
    return main_ui

# Create and display the human review interface
human_review_interface = create_human_review_interface()
display(human_review_interface)

# Export configuration for other boxes
box6_exports = {
    "human_review_enabled": human_review_enabled,
    "auto_approve_threshold": auto_approve_threshold
}

box6_exports_file = Path("./GoogleManusSystem/config/box6_exports.json")
with open(box6_exports_file, "w") as f:
    json.dump(box6_exports, f, indent=2)

print("\n🎉 BOX 6 SETUP COMPLETED SUCCESSFULLY!")
print("👤 Human-in-the-Loop System Initialized")
print(f"🔄 Human Review: {'Enabled' if human_review_enabled else 'Disabled'}")
print(f"🔢 Auto-approve Threshold: {auto_approve_threshold:.2f}")
if server_started:
    print(f"🌐 Web Interface: http://localhost:{server_port}/human-review")